In [2]:
import numpy as np
import pandas as pd

# Using Raw Data
Assumptions: everything counts as debt owned in the state if it is assigned to that state on the certificate <br>
No entries have been cleaned; includes executors, estates, companies, groups, etc.

### Loan Office Certificates 9 States

In [3]:
loans = pd.read_excel("Data/Pre1790/loan_office_certificates_9_states.xlsx")
loans = loans.rename(columns = {"Specie Value ":"Specie Value"})
loans_statewise = loans.groupby(loans["State"])

In [4]:
loans_agg = loans_statewise.agg({"Specie Value": "sum"})
loans_agg["State key"] = ["NH", "MA", "CT", "NY", "NJ", "PA", "DE", "MD", "VA"]
loans_agg

,Specie Value,State key
State,,
1,3.001711e+05,NH
2,2.387481e+06,MA
3,1.275499e+06,CT
4,9.220057e+05,NY
5,6.512044e+05,NJ
6,3.778570e+06,PA
7,6.638712e+04,DE
8,4.047050e+05,MD
9,2.352496e+05,VA


### Liquidated Debt Certificates

In [5]:
def agg_state_debt(state_loans, state):
    state_agg = state_loans.agg({"Dollars": "sum", "90th": "sum"})
    state_agg["Dollars"] += state_agg["90th"] / 90
    state_agg.drop("90th", inplace=True)
    state_agg["State"] = state
    return state_agg

In [6]:
state_liquidated = pd.DataFrame(columns=["State", "Dollars"])
CT_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_CT.xlsx", header=11)
state_liquidated = state_liquidated.append(agg_state_debt(CT_loans, "CT"), ignore_index=True)

DE_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_DE.xlsx", header=10)
state_liquidated = state_liquidated.append(agg_state_debt(DE_loans, "DE"), ignore_index=True)

MA_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_MA.xlsx", header=11)
state_liquidated = state_liquidated.append(agg_state_debt(MA_loans, "MA"), ignore_index=True)

NH_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NH.xlsx", header=11)
state_liquidated = state_liquidated.append(agg_state_debt(NH_loans, "NH"), ignore_index=True)

NJ_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NJ.xlsx", header=10)
NJ_loans = NJ_loans.rename(columns = {"Dollar ":"Dollars"})
state_liquidated = state_liquidated.append(agg_state_debt(NJ_loans, "NJ"), ignore_index=True)

NY_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_NY.xlsx", header=11)
#handle exception at 5164: "22/8"
NY_loans.at[5164, "90th"] = float(22/8)
state_liquidated = state_liquidated.append(agg_state_debt(NY_loans, "NY"), ignore_index=True)

PA_loans1 = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_stelle.xlsx", header=11)
#Note: PA_story also has specie value
PA_loans2 = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_PA_story.xlsx", header=11)
PA_agg1 = PA_loans1.agg({"Dollars": "sum", "90th": "sum"})
PA_agg2 = PA_loans2.agg({"Dollars": "sum", "90th": "sum", "10th": "sum"})
PA_agg2["Dollars"] += PA_agg1["Dollars"] + PA_agg1["90th"] / 90 + PA_agg2["90th"] / 90 + PA_agg2["10th"] / 8
PA_agg2.drop(["90th", "10th"], inplace=True)
PA_agg2["State"] = "PA"
state_liquidated = state_liquidated.append(PA_agg2, ignore_index=True)

RI_loans = pd.read_excel("Data/Pre1790/liquidated_debt_certificates_RI.xlsx", header=11)
RI_agg = RI_loans.agg({"Dollars": "sum", "90th": "sum", "8th": "sum"})
RI_agg["Dollars"] += RI_agg["90th"] / 90 + RI_agg["8th"] / 8
RI_agg.drop(["90th", "8th"], inplace=True)
RI_agg["State"] = "RI"
state_liquidated = state_liquidated.append(RI_agg, ignore_index=True)

C:\Users\alice\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
state_liquidated

,State,Dollars
0,CT,1.354322e+05
1,DE,1.363059e+05
2,MA,4.578235e+05
3,NH,1.382644e+04
4,NJ,1.768007e+06
5,NY,3.649823e+06
6,PA,2.257929e+06
7,RI,1.830005e+05


### Pierce Certificates

In [8]:
Pierce = pd.read_excel("Data/Pre1790/Pierce_Certs_cleaned_2019.xlsx")
Pierce_statewise = Pierce.groupby(Pierce["State"])
Pierce_agg = Pierce_statewise.agg({"Value": "sum"})
Pierce_agg
#CS means Congress, F means foreign

,Value
State,
CS,150384.71
CT,1137988.73
DE,130868.22
F,76532.18
GA,93226.85
MA,2217598.90
MD,550014.80
NC,329382.65
NH,380285.90


### Cumulation

In [9]:
loan_state_merge = loans_agg.merge(state_liquidated, how="outer", right_on="State", left_on="State key")
#make state column uniform
loan_state_merge.at[9.0, "State key"] = "RI"
loan_state_merge.drop("State", axis=1, inplace=True)

loan_state_pierce_merge = loan_state_merge.merge(Pierce_agg, how="outer", right_index=True, left_on="State key")
loan_state_pierce_merge.set_index(loan_state_pierce_merge["State key"], inplace=True)
loan_state_pierce_merge.drop("State key", axis=1, inplace=True)
loan_state_pierce_merge.rename(columns={"Specie Value": "Loan Office Certificates Specie Value",
                                       "Dollars": "State Liquidated Debt Dollars",
                                       "Value": "Pierce Certificates Value"}, inplace=True)
loan_state_pierce_merge

,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value
State key,,,
NH,3.001711e+05,1.382644e+04,380285.90
MA,2.387481e+06,4.578235e+05,2217598.90
CT,1.275499e+06,1.354322e+05,1137988.73
NY,9.220057e+05,3.649823e+06,774934.83
NJ,6.512044e+05,1.768007e+06,513348.62
PA,3.778570e+06,2.257929e+06,1147498.52
DE,6.638712e+04,1.363059e+05,130868.22
MD,4.047050e+05,NaN,550014.80
VA,2.352496e+05,NaN,653914.72


In [10]:
loan_state_pierce_merge.sort_values(["Loan Office Certificates Specie Value", "State Liquidated Debt Dollars", 
                                     "Pierce Certificates Value"], ascending=False, inplace=True)
loan_state_pierce_merge

,Loan Office Certificates Specie Value,State Liquidated Debt Dollars,Pierce Certificates Value
State key,,,
PA,3.778570e+06,2.257929e+06,1147498.52
MA,2.387481e+06,4.578235e+05,2217598.90
CT,1.275499e+06,1.354322e+05,1137988.73
NY,9.220057e+05,3.649823e+06,774934.83
NJ,6.512044e+05,1.768007e+06,513348.62
MD,4.047050e+05,NaN,550014.80
NH,3.001711e+05,1.382644e+04,380285.90
VA,2.352496e+05,NaN,653914.72
DE,6.638712e+04,1.363059e+05,130868.22


In [12]:
loan_state_pierce_merge.to_csv("statewise_debt_aggregation_Pre1790.csv")